In [2]:
from thinkbayes import Pmf
import thinkbayes as tb

In [3]:
pmf = Pmf()
for x in [1,2,3,4,5,6]:
    pmf.Set(x, 1/6)
    

In [4]:
pmf.GetDict()


{1: 0.16666666666666666,
 2: 0.16666666666666666,
 3: 0.16666666666666666,
 4: 0.16666666666666666,
 5: 0.16666666666666666,
 6: 0.16666666666666666}

In [20]:
pmf = Pmf()
text = "This is black and this is blue but this is neither black nur blue"
word_list = [word.lower() for word in text.split()]
for word in word_list:
    pmf.Incr(word, 1)
pmf.GetDict()

{'and': 1,
 'black': 2,
 'blue': 2,
 'but': 1,
 'is': 3,
 'neither': 1,
 'nur': 1,
 'this': 3}